In [ ]:
#############
###paths#####
#############
import sys
import os

from importlib_metadata import distribution
pwd = os.getcwd()
modellingpath = pwd.rpartition("modelling")[0] + pwd.rpartition("modelling")[1] 
sys.path.append(modellingpath + '/lib')
#############

import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


In [ ]:
# Specify name of circuit and variant investigated
circuit_n='circuit14'
variant='2nd'
# Specifiy number of parameter sets in parameterset file to be loaded
n_param_sets = 1000000

print(f'Circuit:{circuit_n}, Variant:{variant}')
# lhs_df = pickle.load( open(modellingpath + '/3954/paper/input/lhs_parameterfiles/df_%s_variant%s_%rparametersets.pkl'%(circuit_n,variant,100000), "rb"))
# lhs_df = pickle.load( open(modellingpath + '/3954/paper/input/lhs_parameterfiles/df_circuit14_variant2nd_100000parametersets', 'rb'))

balanced_lsa_df= pickle.load( open(modellingpath + '/3954/paper/out/analytical/lsa_dataframes/all_dataframes/lsa_df_%s_variant%s_%rparametersets_balanced.pkl'%(circuit_n,variant,n_param_sets), "rb"))
print(balanced_lsa_df['system_class'].value_counts())

variant='fitted2'
n_param_sets = 1000000
# fitDf = pickle.load( open(modellingpath + '/3954/paper/input/fitted_parameterfiles/df_%s_variant%s_%rparametersets.pkl'%(circuit_n,variant,n_param_sets), "rb"))

fitDf = pickle.load( open(modellingpath + '/3954/paper/out/analytical/lsa_dataframes/all_dataframes/lsa_df_%s_variant%s_%sparametersets_balancedSemiBalanced.pkl'%(circuit_n,variant,n_param_sets), "rb"))
print(fitDf['system_class'].value_counts())


In [ ]:
instabilities = ['turing I', 'turing II', 'turing I hopf', 'turing I oscillatory', 'turing II hopf','hopf', 'turing semi-hopf']  
instabilities_df = balanced_lsa_df.loc[balanced_lsa_df['system_class'].isin(instabilities)]

turingStates = ['turing I','turing I oscillatory']  
turing_df = balanced_lsa_df.loc[balanced_lsa_df['system_class'].isin(turingStates)]
zeroSignChanges_df = balanced_lsa_df.loc[balanced_lsa_df['system_class'].isin(['Zero sign changes'])]
turing_df       

In [ ]:
pfitDict = {'Vc': 99.99999998437673,
'Vd': 8.845204237579598,
'Ve': 99.99999902098732,
'Vf': 8.394996554940189,
'Kvd': 0.9400832873750722,
'Kda': 4.142114110177617,
'Kce': 4.342277899710668,
'Kfe': 0.4319218458202641}


paramList = pfitDict.keys()

#copy pandas df to new df
# fitDf = lhs_df.copy()
# for param in paramList:
#     fitDf[param] = pfitDict[param]
# fitDf
# paramList = ['Vc', 'Vd', 'Vf', 'Kvd', 'Kda', 'Kfe','Kce']


In [ ]:
from scipy import stats
def paramsHistogram(balancedDf,specificDf,fitDf,title,crop=2000):
    sns.set(style="white", palette="muted", color_codes=False)

    fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(35,20))
    axs = axs.flatten()
    palette = sns.diverging_palette(10, 220, n=2)
    # paramList=['Dr','VA','VB','VC','VD','VE','VF', 'Kda','Kab', 'Keb', 'Kbd', 'Kfe', 'Kee', 'Kce', 'muLVA']
    df_concat = pd.concat([balancedDf,specificDf,fitDf])
    # df_concat[(np.abs(stats.zscore(df_concat)) < 3).all(axis=1)]
    # df_concat[np.abs(df_concat.Data-df_concat.Data.mean()) <= (3*df_concat.Data.std())]

    for count, parameter in enumerate(paramList):
    # for count, parameter in enumerate(lsa_df.columns[:24]):
        q = df_concat[parameter].quantile(0.3)
        df_concat[df_concat[parameter] < q]
        LogMin, LogMax = np.log10(df_concat.iloc[:,count].min()),np.log10(df_concat.iloc[:,count].max())
        newBins = np.logspace(LogMin/10, LogMax,100)
        sns.distplot(balancedDf.iloc[:crop,count].values,bins=newBins, kde=False, color = 'darkcyan', ax = axs[count], label='Balanced')
        sns.distplot(specificDf.iloc[:crop,count].values,bins=newBins, kde=False, color = 'red', ax = axs[count], label=title)
        sns.distplot(fitDf.iloc[:crop,count].values,bins=newBins, kde=False, color = 'black', ax = axs[count], label='Fit')

        axs[count].set_xscale('log')
        axs[count].set_yticklabels(axs[count].get_yticks(), size = 20)
        axs[count].set_xticklabels(axs[count].get_xticks(), size = 20)

        axs[count].set_title(str(parameter),fontsize=40)
    #     axs[count].set_xlim(0)
        axs[0].legend()
    # fig.legend()
    fig.suptitle(title, fontsize=50)
    fig.tight_layout()
    fig.text(0.5,-0.03, "Parameter Value (log-scale)", ha="center", va="center",  fontsize=40)
    fig.text(-0.03,0.5, "Frequency", ha="center", va="center", rotation=90 ,fontsize=40)
    plt.show()


balanced_lsa_df = balanced_lsa_df[paramList]
instabilities_df = instabilities_df[paramList]
turing_df = turing_df[paramList]
fitDf = fitDf[paramList]

paramsHistogram(balanced_lsa_df,instabilities_df,fitDf, 'Instabilities',crop=10)
# paramsHistogram(lhs_df,balanced_lsa_df,instabilities_df,fitDf, 'Instabilities',crop=100)
# paramsHistogram(lsa_df,turing_df, 'Turing',crop=200)
paramsHistogram(balanced_lsa_df,turing_df,fitDf, 'Turing',crop=10)
# paramsHistogram(lsa_df,zeroSignChanges_df, 'Diffusion independent stability',crop=2000)



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


import pandas as pd
# del concatenated_df
del df
# def concatenate_dataframes(df1, df2, df3,crop=100):
def concatenate_dataframes(df1, df2,crop=100):
    # Create a new column in each data frame to specify the original source
    df1['source'] =  'df1'
    df2['source'] = 'df2'
    # df3['a'] = "blue"
    
    # Concatenate the data frames vertically
    # concatenated_df = pd.concat([df1.iloc[:crop], df2.iloc[:crop], df3.iloc[:crop]], ignore_index=True)
    concatenated_df = pd.concat([df1.iloc[:crop], df2.iloc[:crop]], ignore_index=True)

    return concatenated_df

# concatenated_df = concatenate_dataframes(balanced_lsa_df, instabilities_df, turing_df, crop=5)
df = concatenate_dataframes(balanced_lsa_df, instabilities_df, crop=5)
# concatenated_df

# Create the pairplot with hue and store the AxesSubplot object
sns.pairplot(df, hue="source")

# # Apply logarithmic scale to the axes
# plot.set(xscale="log", yscale="log")

# # Display the plot
plt.show()



In [ ]:
df